In [1]:
import imutils
import argparse
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import time
import cv2
import os
import import_ipynb
from pyimagesearch import CentroidTracker
from pyimagesearch import TrackableObject
import dlib


importing Jupyter notebook from pyimagesearch.ipynb


In [2]:
prototxt = os.path.join("mobilenet_ssd/MobileNetSSD_deploy.prototxt.txt")
model = os.path.join("mobilenet_ssd/MobileNetSSD_deploy.caffemodel")
input_file = os.path.join("videos/example_01.mp4")
# input_file=""
output_file = os.path.join("output/final.mp4")
skip_frames = 30
min_confidence = 0.4

In [3]:
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
    "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
    "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
    "sofa", "train", "tvmonitor"]

# load our serialized model from disk
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(prototxt,model)


[INFO] loading model...


In [4]:
print(len(str(input_file)))
if len(str(input_file))==0:
    print("[INFO] starting video stream...")
    vs = cv2.VideoCapture(0)
    time.sleep(2.0)
# otherwise, grab a reference to the video file
else:
    print("[INFO] opening video file...")
    vs = cv2.VideoCapture(input_file)

21
[INFO] opening video file...


In [5]:
# initialize the video writer (we'll instantiate later if need be)
writer = None

# initialize the frame dimensions (we'll set them as soon as we read the first frame from the video)
W = None
H = None

# instantiate our centroid tracker, then initialize a list to store
# each of our dlib correlation trackers, followed by a dictionary to
# map each unique object ID to a TrackableObject
ct = CentroidTracker(maxDisappeared=40)# maxDistance=50
trackers = []
trackableObjects = {}

# initialize the total number of frames processed thus far, along
# with the total number of objects that have moved either up or down
totalFrames = 0
totalDown = 0
totalUp = 0

# start the frames per second throughput estimator
fps = FPS().start()

In [6]:
while True:
    # grab the next frame and handle if we are reading from either VideoCapture or VideoStream
    frame = vs.read()
    frame = frame[1] if (input_file, False) else frame
    print(frame)
    #print(frame,type(frame))

    # if we are viewing a video and we did not grab a frame then we have reached the end of the video
    if input_file!="" and frame is None:
        break

    #frame=np.array(frame,dtype=object)

    # print("\n\n")
    #print(frame)
    #print(frame[1].shape)
    #print(frame.dtype)
    # resize the frame to have a maximum width of 500 pixels (the
    # less data we have, the faster we can process it), then convert
    # the frame from BGR to RGB for dlib
    
    frame = imutils.resize(frame, width=500)
    #frame = imutils.resize(frame[1], width=500)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    

    # if the frame dimensions are empty, set them
    if W is None or H is None:
        (H, W) = frame.shape[:2]

    # if we are supposed to be writing a video to disk, initialize the writer
    if output_file!="" and writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(output_file, fourcc, 30, (W, H), True)
    
    # initialize the current status along with our list of bounding
    # box rectangles returned by either (1) our object detector or
    # (2) the correlation trackers
    status = "Waiting"
    rects = []

    # check to see if we should run a more computationally expensive object detection method to aid our tracker
    if totalFrames % skip_frames == 0:
        # set the status and initialize our new set of object trackers
        status = "Detecting"
        trackers = []

        # convert the frame to a blob and pass the blob through the network and obtain the detections
        blob = cv2.dnn.blobFromImage(frame, 0.007843, (W, H), 127.5)
        net.setInput(blob)
        detections = net.forward()
        print(f"the detection is {detections} !!!")
        
        # loop over the detections
        for i in np.arange(0, detections.shape[2]):

            # extract the confidence (i.e., probability) associated with the prediction
            confidence = detections[0, 0, i, 2]

            # filter out weak detections by requiring a minimum confidence
            if confidence > min_confidence:

                # extract the index of the class label from the detections list
                idx = int(detections[0, 0, i, 1])

                # if the class label is not a person, ignore it
                if CLASSES[idx] != "person":
                    continue

                # compute the (x, y)-coordinates of the bounding box for the object
                box = detections[0, 0, i, 3:7] * np.array([W, H, W, H])
                (startX, startY, endX, endY) = box.astype("int")

                # construct a dlib rectangle object from the bounding box coordinates and then start the dlib correlation tracker
                tracker = dlib.correlation_tracker()
                rect = dlib.rectangle(startX, startY, endX, endY)
                tracker.start_track(rgb, rect)

                # add the tracker to our list of trackers so we can utilize it during skip frames
                trackers.append(tracker)
    
    # otherwise, we should utilize our object *trackers* rather than object *detectors* to obtain a higher frame processing throughput
    else:

        # loop over the trackers
        for tracker in trackers:

            # set the status of our system to be 'tracking' rather than 'waiting' or 'detecting'
            status = "Tracking"

            # update the tracker and grab the updated position
            tracker.update(rgb)
            pos = tracker.get_position()

            # unpack the position object
            startX = int(pos.left())
            startY = int(pos.top())
            endX = int(pos.right())
            endY = int(pos.bottom())

            # add the bounding box coordinates to the rectangles list
            rects.append((startX, startY, endX, endY))
            
    # draw a horizontal line in the center of the frame -- once an
    # object crosses this line we will determine whether they were
    # moving 'up' or 'down'
    cv2.line(frame, (0, H // 2), (W, H // 2), (0, 255, 255), 1)

    # use the centroid tracker to associate the (1) old object
    # centroids with (2) the newly computed object centroids
    objects = ct.update(rects)

    # loop over the tracked objects
    for (objectID, centroid) in objects.items():

        # check to see if a trackable object exists for the current object ID
        to = trackableObjects.get(objectID, None)

        # if there is no existing trackable object, create one
        if to is None:
            to = TrackableObject(objectID, centroid)

        # otherwise, there is a trackable object so we can utilize it to determine direction
        else:
            # the difference between the y-coordinate of the *current*
            # centroid and the mean of *previous* centroids will tell
            # us in which direction the object is moving (negative for
            # 'up' and positive for 'down')
            y = [c[1] for c in to.centroids]
            direction = centroid[1] - np.mean(y)
            to.centroids.append(centroid)

            # check to see if the object has been counted or not
            if not to.counted:
                # if the direction is negative (indicating the object
                # is moving up) AND the centroid is above the center
                # line, count the object
                if direction < 0 and centroid[1] < H // 2:
#                     playsound('audios/audio.mp3')  #out sound
                    totalUp += 1
                    to.counted = True

                # if the direction is positive (indicating the object
                # is moving down) AND the centroid is below the
                # center line, count the object
                elif direction > 0 and centroid[1] > H // 2:
#                     playsound('audios/In.mp3')
                    totalDown += 1
                    to.counted = True

        # store the trackable object in our dictionary
        trackableObjects[objectID] = to

        # draw both the ID of the object and the centroid of the object on the output frame
        text = "ID {}".format(objectID)
        cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.circle(frame, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)

    # construct a tuple of information we will be displaying on the frame
    info = [
        ("Up", totalUp),
        ("Down", totalDown),
        ("Total",totalUp+totalDown),
        ("Status", status)
    ]

    # loop over the info tuples and draw them on our frame
    for (i, (k, v)) in enumerate(info):
        text = "{}: {}".format(k, v)
        cv2.putText(frame, text, (10, H - ((i * 20) + 20)),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # check to see if we should write the frame to disk
    if writer is not None:
        writer.write(frame)

    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

    # increment the total number of frames processed thus far and then update the FPS counter
    totalFrames += 1
    fps.update()


[[[151 148 149]
  [149 146 147]
  [147 144 145]
  ...
  [166 165 167]
  [167 167 167]
  [164 164 164]]

 [[151 148 149]
  [149 146 147]
  [147 144 145]
  ...
  [157 156 158]
  [156 156 156]
  [154 154 154]]

 [[151 148 149]
  [149 146 147]
  [147 144 145]
  ...
  [178 177 179]
  [174 174 174]
  [173 173 173]]

 ...

 [[148 144 147]
  [148 144 147]
  [147 143 146]
  ...
  [120 122 124]
  [111 111 111]
  [105 105 105]]

 [[148 144 147]
  [148 144 147]
  [147 143 146]
  ...
  [116 118 120]
  [111 111 111]
  [105 105 105]]

 [[148 144 147]
  [148 144 147]
  [147 143 146]
  ...
  [111 113 115]
  [111 111 111]
  [105 105 105]]]
the detection is [[[[0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0.

[[[145 144 146]
  [143 142 144]
  [139 138 140]
  ...
  [186 185 187]
  [188 188 188]
  [187 187 187]]

 [[145 144 146]
  [143 142 144]
  [139 138 140]
  ...
  [192 191 193]
  [189 189 189]
  [189 189 189]]

 [[146 145 147]
  [143 142 144]
  [139 138 140]
  ...
  [187 186 188]
  [187 187 187]
  [188 188 188]]

 ...

 [[149 146 147]
  [147 144 145]
  [147 144 145]
  ...
  [ 99 101 103]
  [ 95  98  98]
  [ 93  96  96]]

 [[151 146 147]
  [149 144 145]
  [149 144 145]
  ...
  [ 97  99 101]
  [ 94  97  97]
  [ 92  95  95]]

 [[151 146 147]
  [149 144 145]
  [149 144 145]
  ...
  [ 97  99 101]
  [ 95  98  98]
  [ 93  96  96]]]
[[[145 144 146]
  [143 142 144]
  [139 138 140]
  ...
  [183 186 186]
  [187 187 187]
  [186 186 186]]

 [[145 144 146]
  [143 142 144]
  [139 138 140]
  ...
  [185 188 188]
  [188 188 188]
  [188 188 188]]

 [[146 145 147]
  [143 142 144]
  [139 138 140]
  ...
  [190 193 193]
  [190 190 190]
  [192 192 192]]

 ...

 [[149 146 147]
  [147 144 145]
  [146 143 144]
  ..

[[[145 141 144]
  [141 137 140]
  [140 136 139]
  ...
  [183 183 183]
  [183 183 183]
  [183 183 183]]

 [[145 141 144]
  [142 138 141]
  [141 137 140]
  ...
  [186 186 186]
  [186 186 186]
  [185 185 185]]

 [[144 140 143]
  [142 138 141]
  [141 137 140]
  ...
  [189 189 189]
  [189 189 189]
  [187 187 187]]

 ...

 [[146 146 146]
  [144 144 144]
  [146 146 146]
  ...
  [ 98  98  98]
  [ 90  93  93]
  [ 79  82  82]]

 [[146 146 146]
  [146 146 146]
  [146 146 146]
  ...
  [ 98  98  98]
  [ 90  93  93]
  [ 79  82  82]]

 [[145 145 145]
  [144 144 144]
  [144 144 144]
  ...
  [100 100 100]
  [ 91  94  94]
  [ 82  85  85]]]
[[[146 142 145]
  [144 140 143]
  [144 140 143]
  ...
  [182 182 182]
  [183 183 183]
  [183 183 183]]

 [[146 142 145]
  [144 140 143]
  [144 140 143]
  ...
  [186 186 186]
  [186 186 186]
  [185 185 185]]

 [[145 141 144]
  [142 138 141]
  [142 138 141]
  ...
  [189 189 189]
  [189 189 189]
  [187 187 187]]

 ...

 [[146 146 146]
  [144 144 144]
  [146 146 146]
  ..

[[[147 144 145]
  [144 141 142]
  [145 142 143]
  ...
  [182 185 185]
  [183 183 183]
  [183 183 183]]

 [[147 144 145]
  [144 141 142]
  [145 142 143]
  ...
  [182 185 185]
  [185 185 185]
  [186 186 186]]

 [[146 143 144]
  [144 141 142]
  [145 142 143]
  ...
  [184 187 187]
  [188 188 188]
  [187 187 187]]

 ...

 [[146 146 146]
  [145 145 145]
  [147 147 147]
  ...
  [ 95  98  98]
  [ 91  94  94]
  [ 77  80  80]]

 [[145 145 145]
  [145 145 145]
  [145 145 145]
  ...
  [ 97 100  98]
  [ 88  91  91]
  [ 76  79  79]]

 [[145 145 145]
  [145 145 145]
  [144 144 144]
  ...
  [ 99 102 100]
  [ 90  93  93]
  [ 79  82  82]]]
[[[149 146 147]
  [147 144 145]
  [146 143 144]
  ...
  [185 185 185]
  [185 185 185]
  [185 185 185]]

 [[148 145 146]
  [146 143 144]
  [148 145 146]
  ...
  [185 185 185]
  [185 185 185]
  [186 186 186]]

 [[147 144 145]
  [145 142 143]
  [148 145 146]
  ...
  [187 187 187]
  [188 188 188]
  [187 187 187]]

 ...

 [[146 146 146]
  [145 145 145]
  [147 147 147]
  ..

[[[149 146 147]
  [148 145 146]
  [147 144 145]
  ...
  [186 186 186]
  [186 186 186]
  [186 186 186]]

 [[151 148 149]
  [148 145 146]
  [147 144 145]
  ...
  [188 188 188]
  [188 188 188]
  [187 187 187]]

 [[149 146 147]
  [148 145 146]
  [147 144 145]
  ...
  [182 182 182]
  [179 179 179]
  [179 179 179]]

 ...

 [[145 145 145]
  [146 146 146]
  [144 144 144]
  ...
  [ 94  96  98]
  [ 88  91  91]
  [ 79  82  82]]

 [[144 144 144]
  [144 144 144]
  [145 145 145]
  ...
  [ 97  99 101]
  [ 91  94  94]
  [ 82  85  85]]

 [[143 143 143]
  [143 143 143]
  [143 143 143]
  ...
  [ 98 100 102]
  [ 92  95  95]
  [ 84  87  87]]]
[[[149 146 147]
  [148 145 146]
  [147 144 145]
  ...
  [186 186 186]
  [186 186 186]
  [186 186 186]]

 [[151 148 149]
  [148 145 146]
  [147 144 145]
  ...
  [189 189 189]
  [188 188 188]
  [187 187 187]]

 [[151 148 149]
  [148 145 146]
  [147 144 145]
  ...
  [186 186 186]
  [185 185 185]
  [183 183 183]]

 ...

 [[145 145 145]
  [145 145 145]
  [142 142 142]
  ..

...

[[[152 149 150]
  [151 148 149]
  [153 150 151]
  ...
  [188 188 188]
  [189 189 189]
  [188 188 188]]

 [[152 149 150]
  [152 149 150]
  [154 151 152]
  ...
  [188 188 188]
  [188 188 188]
  [187 187 187]]

 [[154 151 152]
  [154 151 152]
  [156 153 154]
  ...
  [187 187 187]
  [188 188 188]
  [187 187 187]]

 ...

 [[137 134 135]
  [135 132 133]
  [138 135 136]
  ...
  [118 120 122]
  [113 115 117]
  [104 106 108]]

 [[140 137 138]
  [135 132 133]
  [135 132 133]
  ...
  [126 128 130]
  [121 123 125]
  [112 114 116]]

 [[151 148 149]
  [144 141 142]
  [140 137 138]
  ...
  [132 134 136]
  [130 132 134]
  [125 127 129]]]
[[[151 151 151]
  [149 149 149]
  [150 150 150]
  ...
  [188 188 188]
  [189 189 189]
  [188 188 188]]

 [[151 151 151]
  [150 150 150]
  [152 152 152]
  ...
  [188 188 188]
  [188 188 188]
  [187 187 187]]

 [[152 152 152]
  [151 151 151]
  [153 153 153]
  ...
  [187 187 187]
  [188 188 188]
  [187 187 187]]

 ...

 [[137 134 135]
  [135 132 133]
  [138 135 136]
  ..

[[[152 152 152]
  [152 152 152]
  [151 151 151]
  ...
  [185 187 189]
  [188 188 188]
  [189 189 189]]

 [[152 152 152]
  [152 152 152]
  [151 151 151]
  ...
  [184 186 188]
  [188 188 188]
  [187 187 187]]

 [[152 152 152]
  [152 152 152]
  [152 152 152]
  ...
  [183 185 187]
  [185 185 185]
  [183 183 183]]

 ...

 [[143 143 143]
  [138 138 138]
  [132 132 130]
  ...
  [132 132 132]
  [125 127 129]
  [118 120 122]]

 [[150 150 150]
  [144 144 144]
  [138 138 136]
  ...
  [137 137 137]
  [132 134 136]
  [129 131 133]]

 [[149 149 149]
  [147 147 147]
  [140 140 138]
  ...
  [138 138 138]
  [137 139 141]
  [135 137 139]]]
the detection is [[[[0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0.

[[[154 151 152]
  [152 149 150]
  [153 150 151]
  ...
  [189 189 189]
  [188 188 188]
  [188 188 188]]

 [[151 148 149]
  [148 145 146]
  [149 146 147]
  ...
  [189 189 189]
  [188 188 188]
  [188 188 188]]

 [[151 148 149]
  [148 145 146]
  [149 146 147]
  ...
  [188 188 188]
  [187 187 187]
  [187 187 187]]

 ...

 [[136 135 137]
  [136 135 137]
  [135 135 135]
  ...
  [126 128 130]
  [121 123 125]
  [109 111 113]]

 [[142 141 143]
  [137 136 138]
  [136 136 136]
  ...
  [136 135 137]
  [129 131 133]
  [122 124 126]]

 [[146 145 147]
  [143 142 144]
  [138 138 138]
  ...
  [136 135 137]
  [134 136 138]
  [130 132 134]]]
[[[153 150 151]
  [152 149 150]
  [152 149 150]
  ...
  [189 189 189]
  [188 188 188]
  [188 188 188]]

 [[151 148 149]
  [149 146 147]
  [151 148 149]
  ...
  [190 190 190]
  [189 189 189]
  [188 188 188]]

 [[152 149 150]
  [149 146 147]
  [151 148 149]
  ...
  [187 187 187]
  [189 189 189]
  [188 188 188]]

 ...

 [[136 135 137]
  [136 135 137]
  [135 135 135]
  ..

[[[151 151 151]
  [151 151 151]
  [153 150 151]
  ...
  [188 188 188]
  [188 188 188]
  [188 188 188]]

 [[151 151 151]
  [150 150 150]
  [153 150 151]
  ...
  [188 188 188]
  [187 187 187]
  [187 187 187]]

 [[151 151 151]
  [151 151 151]
  [153 150 151]
  ...
  [187 187 187]
  [187 187 187]
  [187 187 187]]

 ...

 [[138 137 139]
  [136 135 137]
  [136 136 136]
  ...
  [119 118 120]
  [111 116 116]
  [104 109 109]]

 [[139 138 140]
  [137 136 138]
  [133 133 133]
  ...
  [129 128 130]
  [119 124 124]
  [112 117 117]]

 [[147 146 148]
  [142 141 143]
  [138 138 138]
  ...
  [138 137 139]
  [132 137 137]
  [128 133 133]]]
[[[149 149 149]
  [149 149 149]
  [151 151 151]
  ...
  [183 186 186]
  [187 187 187]
  [188 188 188]]

 [[149 149 149]
  [149 149 149]
  [151 151 151]
  ...
  [184 187 187]
  [188 188 188]
  [188 188 188]]

 [[150 150 150]
  [150 150 150]
  [150 150 150]
  ...
  [186 186 186]
  [188 188 188]
  [187 187 187]]

 ...

 [[138 137 139]
  [136 135 137]
  [137 136 138]
  ..

[[[151 151 151]
  [152 152 152]
  [152 152 152]
  ...
  [184 187 187]
  [189 189 189]
  [190 190 190]]

 [[150 150 150]
  [149 149 149]
  [153 153 153]
  ...
  [184 187 187]
  [189 189 189]
  [189 189 189]]

 [[151 151 151]
  [149 149 149]
  [151 151 151]
  ...
  [186 186 186]
  [188 188 188]
  [187 187 187]]

 ...

 [[137 136 138]
  [136 135 137]
  [135 134 136]
  ...
  [118 120 122]
  [112 115 115]
  [108 111 111]]

 [[143 142 144]
  [139 138 140]
  [137 136 138]
  ...
  [124 128 130]
  [121 124 124]
  [116 119 119]]

 [[146 145 147]
  [144 143 145]
  [140 139 141]
  ...
  [128 132 134]
  [130 133 133]
  [128 131 131]]]
[[[152 152 152]
  [151 151 151]
  [151 151 151]
  ...
  [184 187 187]
  [189 189 189]
  [190 190 190]]

 [[150 150 150]
  [150 150 150]
  [153 153 153]
  ...
  [184 187 187]
  [189 189 189]
  [190 190 190]]

 [[151 151 151]
  [149 149 149]
  [152 152 152]
  ...
  [187 187 187]
  [189 189 189]
  [188 188 188]]

 ...

 [[137 136 138]
  [137 136 138]
  [137 136 138]
  ..

In [7]:
# stop the timer and display FPS information
fps.stop()
print("[INFO] elapsed time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

# check to see if we need to release the video writer pointer
if writer is not None:
    writer.release()

# if we are not using a video file, stop the camera video stream
if not (input_file!=""):
    vs.release()

# otherwise, release the video file pointer
else:
    vs.release()

# close any open windows
cv2.destroyAllWindows()



[INFO] elapsed time: 49.19
[INFO] approx. FPS: 20.33


In [13]:
import numpy as np
a=np.array([[[1,2,3,4],[2,3,45,5]],[[1,2,3,4],[2,2,2,1]]],np.int32)
a.shape
a[0,1,2]

45

In [16]:
centorid=np.zeros((4,2),dtype="int")

In [19]:
centorid[0]=[4,5]
centorid

array([[4, 5],
       [0, 0],
       [0, 0],
       [0, 0]])

In [20]:
from scipy.spatial import distance as dist
import numpy as np
np.random.seed(42)
objectCentroids = np.random.uniform(size=(2, 2))
print("OC:\n",objectCentroids)
centroids = np.random.uniform(size=(3, 2))
print("C:\n",centroids)
D = dist.cdist(objectCentroids, centroids)
print("D:\n",D)

OC:
 [[0.37454012 0.95071431]
 [0.73199394 0.59865848]]
C:
 [[0.15601864 0.15599452]
 [0.05808361 0.86617615]
 [0.60111501 0.70807258]]
D:
 [[0.82421549 0.32755369 0.33198071]
 [0.72642889 0.72506609 0.17058938]]
